### temporary setup after `make ssh`, `cdata`, and `make syncup`

```python
mamba activate fastai2
mamba install cython -y
pip install "git+https://github.com/philferriere/cocoapi.git#egg=pycocotools&subdirectory=PythonAPI"
pip install git+git://github.com/waspinator/pycococreator.git@0.2.0
cd work
pip install -e ceruleanml # install local ceruleanml package after deps installed with conda
mamba deactivate
```

In [1]:
import ceruleanml.data as data
from pathlib import Path
import skimage.io as skio
import os

ml_data_path = os.path.join(os.path.abspath(os.getcwd()),"../../data/cerulean_v2_example/")
path = Path(ml_data_path)
layer_pths = list(path.glob("assets_test1_png/*"))
outdir = "../../data/cerulean_v2_example/tiled_data_example"
os.makedirs(outdir, exist_ok=True)

In [2]:
# img_fname = layer_pths[1]
# arr = skio.imread(layer_pths[1])
# tiled_arr = data.reshape_split(arr, (512, 512))

In [3]:
info = {
    "description": "Cerulean Dataset V2",
    "url": "none",
    "version": "1.0",
    "year": 2021,
    "contributor": "Skytruth",
    "date_created": "2022/2/23"
}

licenses = [
    {
        "url": "none",
        "id": 1,
        "name": "CeruleanDataset V2"
    }
]
categories = [{"supercategory":"slick", "id":1,"name":"infra_slick"},
              {"supercategory":"slick", "id":2,"name":"natural_seep"},
              {"supercategory":"slick", "id":3,"name":"coincident_vessel"},
              {"supercategory":"slick", "id":4,"name":"recent_vessel"},
              {"supercategory":"slick", "id":5,"name":"old_vessel"},
              {"supercategory":"slick", "id":6,"name":"ambiguous"}]

coco_output = {
    "info": info,
    "licenses": licenses,
    "images": [],
    "annotations": [],
    "categories": categories
}

In [4]:
data.create_coco_from_photopea_layers(layer_pths, outdir, coco_output)

finished saving 117 images
